# Zero Shot with Analysis Info: EJB Remote - Rule: remote-ejb-to-quarkus-0000

> This notebook will work with a LLM to generate a fix for a EJB Remote to REST API related rule found in 'ejb-remote'.  We only have 1 sample application for this notebook.

##### Sample Applications Used
* 1 sample app from [JBoss EAP Quickstarts](https://github.com/jboss-developer/jboss-eap-quickstarts/tree/7.4.x):  ejb-remote
* Christopher May [christophermay07](https://github.com/christophermay07) migrated this application to Quarkus in https://github.com/christophermay07/quarkus-migrations.  
* We have created new repositories that contain the original Java EE state in the 'main' branch and Chris' Quarkus migration in the 'quarkus' branch 
    * [ejb-remote](https://github.com/konveyor-ecosystem/ejb-remote)

##### Using Custom Rules for EJB Remote
* Rules were developed by Juanma [@jmle](https://github.com/jmle)
    * Rules originally from: https://github.com/jmle/kai-examples/blob/main/rules/01-remote-ejb-to-quarkus.windup.yaml

In [1]:
# Install local kai package in the current Jupyter kernel
import sys

!{sys.executable} -m pip install -e ../../

Obtaining file:///Users/jmatthews/git/jwmatthews/kai
  Preparing metadata (setup.py) ... done
  Attempting uninstall: kai
    Found existing installation: kai 0.0.1
    Uninstalling kai-0.0.1:
      Successfully uninstalled kai-0.0.1
  Running setup.py develop for kai


In [2]:
# Bring in the common deps
import os
import pprint

from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

pp = pprint.PrettyPrinter(indent=2)


def get_java_in_result(text: str):
    languages = ["java", "xml"]
    for lang in languages:
        _, updated_file = text.split(f"## Updated File")
        try:
            _, after = updated_file.split(f"```{lang}")
            return after.split("```")[0]
        except ValueError:
            # ValueError: not enough values to unpack (expected 2, got 1)
            print(f"Didn't find a result for language '{lang}'")
    print(f"Unable to find any Java or XML in the result:\n {text}")
    return "Failed to extract updated file, this is an error in our client logic not yet handling all edge cases.  We don't consider this a problem with the LLM."


def write_output_to_disk(
    out_dir,
    formatted_template,
    example_javaee_file_contents,
    example_quarkus_file_contents,
    result,
):
    try:
        os.makedirs(out_dir, exist_ok=True)
    except OSError as error:
        print(f"Error creating directory {out_dir}: {error}")
        raise error

    # Save the template to disk
    template_path = os.path.join(out_dir, "template.txt")
    with open(template_path, "w") as f:
        f.truncate()
        f.write(formatted_template)
    print(f"Saved template to {template_path}")

    # Save the result
    result_path = os.path.join(out_dir, "result.txt")
    with open(result_path, "w") as f:
        f.truncate()
        f.write(result)
    print(f"Saved result to {result_path}")

    # Extract the updated java code and save it
    updated_file_contents = get_java_in_result(result)
    updated_file_path = os.path.join(out_dir, "updated_file.java")
    with open(updated_file_path, "w") as f:
        f.truncate()
        f.write(updated_file_contents)
    print(f"Saved updated java file to {updated_file_path}")

    # Save the original source code
    original_file_path = os.path.join(out_dir, "original_file.java")
    with open(original_file_path, "w") as f:
        f.truncate()
        f.write(example_javaee_file_contents)
    print(f"Saved original java file to {original_file_path}")

    # Save the Quarkus version we already in Git to use as a comparison
    known_quarkus_file_path = os.path.join(out_dir, "quarkus_version_from_git.java")
    with open(known_quarkus_file_path, "w") as f:
        f.truncate()
        f.write(example_quarkus_file_contents)
    print(f"Saved Quarkus version from Git to {known_quarkus_file_path}")

# Create a Prompt
## Step #1:  Create a Prompt Template


In [3]:
template_with_solved_example_and_analysis = """
    # Java EE to Quarkus Migration
    You are an AI Assistant trained on migrating enterprise JavaEE code to Quarkus.
    I will give you an example of a JavaEE file and you will give me the Quarkus equivalent.

    To help you update this file to Quarkus I will provide you with static source code analysis information
    highlighting an issue which needs to be addressed.

    Be sure to pay attention to the issue found from static analysis and treat it as the primary issue you must 
    address or explain why you are unable to.

    Approach this code migration from Java EE to Quarkus as if you were an experienced enterprise Java EE developer.
    Before attempting to migrate the code to Quarkus, explain each step of your reasoning through what changes 
    are required and why. 

    Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes 
    to imports we need to consider.

    As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.
    
    After you have shared your step by step thinking, provide a full output of the updated file:

    # Input information
    ## Issue found from static code analysis of the Java EE code which needs to be fixed to migrate to Quarkus
    Issue to fix:  "{analysis_message}"

    ## Input file name
    Filename: "{src_file_name}"

    ## Input Line number of the issue first appearing in the Java EE code source code example below
    Line number: {analysis_lineNumber}
    
    ## Input source code file contents for "{src_file_name}"
    ```java 
    {src_file_contents}
    ```

    # Output Instructions
    Structure your ouput in Markdown format such as:

    ## Reasoning
    Write the step by step reasoning in this markdown section.
    If you are unsure of a step or reasoning, clearly state you are unsure and why.

    ## Updated File
    ```java
        Write the updated file for Quarkus in this section
    ```
   
    """

## Step #2: Load the analysis report

In [4]:
from kai.report import Report

ruleset_name = "kai/quarkus"
rule_name = "remote-ejb-to-quarkus-00000"

# Static code analysis was run prior and committed to this repo
path_ejb_remote_analysis = "./analysis_report/ejb-remote/output.yaml"

# We will assume that ejb_remote is the app we want to migrate
# We will assume that we do NOT have a solved example for this example

ejb_remote_report = Report(path_ejb_remote_analysis)

Reading report from ./analysis_report/ejb-remote/output.yaml


## Step #3 Define a function to extract the analysis incident metadata

In [5]:
def get_incident_data(report, ruleset_name, rule):
    return report.report[ruleset_name]["violations"][rule]

## Step #4: Setup access to the source code git repos


In [6]:
from kai.scm import GitDiff

##
# The App to MIGRATE will be ejb_remote
ejb_remote_src_path = "../../samples/sample_repos/ejb-remote"
ejb_remote_diff = GitDiff(ejb_remote_src_path)
# Ensure we checked out the 'quarkus' branch of the app's repo
# we will use the quarkus branch at end to compare to what the 'answer' is essentially
# we don't use it with the LLM, just as a data point for comparison after
ejb_remote_diff.checkout_branch("quarkus")


## Step 5: Create the template args we will pass into our prompt


In [7]:
src_report = ejb_remote_report
src_diff = ejb_remote_diff
src_incident = get_incident_data(src_report, ruleset_name, rule_name)

# We assume that we have at least one incident for each rule
# will only use the first and ignore extras
src_entry = src_incident["incidents"][0]
print(f"Using src incident: {src_entry}")
src_file_name = src_report.get_cleaned_file_path(src_entry["uri"])
src_file_contents = src_diff.get_file_contents(src_file_name, "main")
analysis_message = src_entry["message"]
###
# Note that for a match of 'persistence-to-quarkus-00000' we will NOT have a lineNumber
###
analysis_lineNumber = src_entry.get("lineNumber", "")

template_args = {
    "src_file_name": src_file_name,
    "src_file_contents": src_file_contents,
    "analysis_message": analysis_message,
    "analysis_lineNumber": analysis_lineNumber,
}

Using src incident: {'uri': 'file:///tmp/source-code/server-side/src/main/java/org/jboss/as/quickstarts/ejb/remote/stateful/CounterBean.java', 'message': 'Remote EJBs are not supported in Quarkus, and therefore its use must be removed and replaced with REST functionality. In order to do this:\n 1. Replace the `@Remote` annotation on the class with a `@jakarta.ws.rs.Path("<endpoint>")` annotation. An endpoint must be added to the annotation in place of `<endpoint>` to specify the actual path to the REST service.\n 2. Remove `@Stateless` annotations if present. Given that REST services are stateless by nature, it makes it unnecessary.\n 3. For every public method on the EJB being converted, do the following:\n - Annotate the method with `@jakarta.ws.rs.GET`\n - Annotate the method with `@jakarta.ws.rs.Path("<endpoint>")` and give it a proper endpoint path. As a rule of thumb, the method name can be used as endpoint, for instance:\n ```\n @Path("/increment")\n public void increment() \n `

# Step 6: Run through the incidents we have to generate a fix


In [8]:
model_name = "gpt-4-1106-preview"
# model_name = "gpt-3.5-turbo-16k"
llm = ChatOpenAI(temperature=0.1, model_name=model_name)


print(f"Processing rule: {rule_name}")
formatted_prompt = template_with_solved_example_and_analysis.format(**template_args)
prompt = PromptTemplate.from_template(template_with_solved_example_and_analysis)
chain = LLMChain(llm=llm, prompt=prompt)
result = chain.run(template_args)

src_file_name = template_args["src_file_name"]
src_file_contents = template_args["src_file_contents"]
# For comparison, we will look up what the source file looks like from Quarkus branch
# this serves as a way of comparing to what the 'answer' is that was done manually by a knowledgeable developer
try:
    src_file_from_quarkus_branch = src_diff.get_file_contents(src_file_name, "quarkus")
except:
    # For some cases we will be removing the original file from the quarkus branch
    # and creating a different named file, so this naieve approach won't work
    # of grabbing the same filename from quarkus branch and assume that 
    # is the fixed version for comparison
    src_file_from_quarkus_branch = ""

output_dir = f"output/{model_name}/ejb_remote/{ruleset_name}/{rule_name}/zero_shot_with_analysis/"
write_output_to_disk(
    output_dir,
    formatted_prompt,
    src_file_contents,
    src_file_from_quarkus_branch,
    result,
)
print(f"Completed processing of rule: {rule_name}")

/Users/jmatthews/git/jwmatthews/kai/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/jmatthews/git/jwmatthews/kai/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Processing rule: remote-ejb-to-quarkus-00000
Saved template to output/gpt-4-1106-preview/ejb_remote/kai/quarkus/remote-ejb-to-quarkus-00000/few_shot/template.txt
Saved result to output/gpt-4-1106-preview/ejb_remote/kai/quarkus/remote-ejb-to-quarkus-00000/few_shot/result.txt
Saved updated java file to output/gpt-4-1106-preview/ejb_remote/kai/quarkus/remote-ejb-to-quarkus-00000/few_shot/updated_file.java
Saved original java file to output/gpt-4-1106-preview/ejb_remote/kai/quarkus/remote-ejb-to-quarkus-00000/few_shot/original_file.java
Saved Quarkus version from Git to output/gpt-4-1106-preview/ejb_remote/kai/quarkus/remote-ejb-to-quarkus-00000/few_shot/quarkus_version_from_git.java
Completed processing of rule: remote-ejb-to-quarkus-00000
